In [1]:
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import time
start = time.time()

In [2]:
with open ('training/labels.tsv', 'r') as f:
    txt = f.read()

In [3]:
range_of_posts = re.findall('[0-9]{6}', txt)
# txt

In [4]:
post_filenames = ["training/posts/post-{}.xml".format(id) for id in range_of_posts]

In [5]:
# post_filenames[0:10]

In [6]:
def return_elements(filename):
    # get board name
    with open(filename, 'r') as f:
        txt = f.read()
    soup = BeautifulSoup(txt, 'lxml')
    message = soup.message
    board_id = message.board_id.text
    post_time = message.post_time.text
    subject = message.subject.text
    post_id = message.id.text
    author_id = message.author.get('href').split('/')[-1]
    root_id = message.root.get('href').split('/')[-1]
    body_text, emoji_ids = parse_body(message) # what to get out of body
    d = {}
#     d[author_id] = 
    return [author_id, board_id, post_time, subject, post_id, root_id, body_text, emoji_ids]

def parse_body(message):
    counter = 0
    for content in message.contents:
        if content.name == 'last_edit_time':
            break
        counter += 1
    body = message.contents[counter+1]
    body_text = re.findall("[a-zA-Z0-9]+(?![^<]*\>)", body.string)
    body_text = list(filter(lambda a: a != 'nbsp', body_text))
    emojis = re.findall('id="[a-z]+"', body.string)
    emoji_ids = [emoji.split('"')[1] for emoji in emojis]
    if len(emoji_ids) == 0:
        emoji_ids = None
    return body_text, emoji_ids
        

In [7]:
list_of_parsings = [return_elements(filename) for filename in post_filenames]
df = pd.DataFrame(list_of_parsings)
df.columns = ['author_id', 'board_id', 'post_time', 'subject', 'post_id', 'root_id', 'body_text', 'emoji_ids']

In [8]:
df.head()
end = time.time()
# df.to_csv('training_df.csv')
print('{} seconds have passsed'.format((end-start)))

3.1114749908447266 seconds have passsed


## making sure the body is parsed correctly


In [9]:
with open("training/posts/post-135294.xml") as f:
    post = f.read()

In [10]:
soup = BeautifulSoup(post, 'lxml')
message = soup.message

In [11]:
print(' '.join(parse_body(message)[0]))
print(message.contents[14])

Thanks Sophie RO sometimes this thread is the only thing keeping me moving forward Sometimes the only reason I am able to see or even notice something has bee on my mind is when I wite it here It seems to be a safe space for me Really the only thing I m doing to try and work through it is trying to manage it with some breathing distraction if possible at work Its hard though because some days I just need to work beside someone to help calm the anxiety where as other times I know I just need to be alone And when I m trying so hard not to let myself go into panic mode it really drains my energy At first thought there isn t anyone I d be feel ok to alk about this with I mean with my colleagues I don t know them well enough to know if that s something I could ask them But also it would kinda feel weird Going back to the GP early I don t know I haven considered it but it s not realy something I want to do but I guess if it came down to it I would Making a thread seems fine but I know it s i